In [88]:
import requests
from bs4 import BeautifulSoup
import time
import re
import json

In [89]:
def search(q):
    q = q.replace('\n', '').replace('（', '(').replace('）', ')')
    print("START : " + q)
    q = re.sub("\(.+\)$", "", " ".join(q.split()[:-3]), re.UNICODE)
    query = re.sub('(!|\u3000|/|\s|>|<|\.)+', "%20", q)
    url_search = 'https://eiga.com/search/' + query
    res_search = requests.get(url_search )
    res_search.encoding = res_search.apparent_encoding
    soup_search = BeautifulSoup(res_search.content, "lxml")
    result =  soup_search.find('section', attrs={"id": "rslt-movie"})
    if(result != None):
        path = result.find('li', attrs={"class": "col-s-3"}).find('a')["href"]
        url_review = 'https://eiga.com' + path
        return url_review
    else:
        print("**************************************************")
        print(q + " HAS NO RESULT")
        print("**************************************************")
        return "error"

In [110]:
def scrape(query):
    page_num = 1
    data = {
        "id": -1,
        "rating": -1,
        "check-in":-1,
        "review-count":-1
    }
    url_review=search(query)
    
    if(url_review == "error"):
        return "error"
    

    res = requests.get(url_review)
    res.encoding = res.apparent_encoding
    soup = BeautifulSoup(res.content, "lxml")
    
    rating = soup.find('span', attrs={"class": "rating-star"})
    review_count = soup.find('span', attrs={"itemprop": "reviewCount"})
    check_in = soup.find('a', attrs={"class": "icon-movie-checkin"}).find('strong')
    data["rating"] = 0 if rating.text == '－' else float(rating.text)
    data["review-count"] = 0 if review_count is None else int(review_count.text)
    data["check-in"] = 0 if check_in is None else int(check_in.text)
        
    return data

In [111]:
input_file = '../2018_movie_clean'

In [112]:
num = 1
for q in open(input_file, 'r', encoding='utf-8').readlines():
    output_file = './{0}.json'.format(num)
    if(num <= 700):
        num += 1
        continue
    
    print(num)
    with open(output_file, 'w') as f:
        data = scrape(q)
        if(data == "error"):
            num += 1
            continue
        data["id"] = num
        jsn =  json.dumps(data,ensure_ascii=False, indent=2) 
        f.write(jsn)   
    f.close()
    num += 1
    time.sleep(1)

699
START : 心魔師	2018	10	27
<!DOCTYPE html>
<html lang="ja"> <head> <meta content="ie=edge" http-equiv="x-ua-compatible"/> <meta content="width=1060" name="viewport"/> <meta charset="utf-8"/> <title>心魔師 : 作品情報 - 映画.com</title> <meta content="心魔師の作品情報。上映スケジュール、映画レビュー、予告動画。日本と中国が手を組んで新たな才能を発掘する、日中若手映画人合同プロジェクト第1弾作品として製作されたサイコサスペン..." name="description"/> <meta content="心魔師,情報,映画" name="keywords"/> <meta content="app-id=382023297" name="apple-itunes-app"/> <meta content="summary" name="twitter:card"/> <meta content="@eigacom" name="twitter:site"/> <meta content="ja_JP" property="og:locale"/> <meta content="心魔師 : 作品情報 - 映画.com" property="og:title"/> <meta content="article" property="og:type"/> <meta content="https://eiga.com/movie/89422/" property="og:url"/> <meta content="https://eiga.k-img.com/images/movie/89422/photo/b92bab46bacf0789/640.jpg" property="og:image"/> <meta content="心魔師の作品情報。上映スケジュール、映画レビュー、予告動画。日本と中国が手を組んで新たな才能を発掘する、日中若手映画人合同プロジェクト第1弾作品として製作されたサイコサスペン..." property="og:descr

700
START : SAVE HENOKO	2018	10	27
**************************************************
SAVE HENOKO HAS NO RESULT
**************************************************
701
START : 第三世代	2018	10	27
<html>
<head><title>504 Gateway Time-out</title></head>
<body bgcolor="white">
<center><h1>504 Gateway Time-out</h1></center>
<hr/><center>nginx</center>
</body>
</html>



AttributeError: 'NoneType' object has no attribute 'find'

In [105]:
scrape("心魔師	2018	10	27")

START : 心魔師	2018	10	27


{'id': -1, 'rating': 3.0, 'check-in': 347, 'review-count': 4}